# Semesterarbeit Scientific Programming

## Libraries and settings

In [2]:
# Libraries
import os
import pandas as pd
import re
import sqlite3
import pycountry 
import matplotlib.pyplot as plt
import numpy as np
from scipy.stats import ttest_ind
import streamlit as st

c:\Users\pasca\OneDrive - ZHAW\FS23\scientific_programming\Semesterarbeit


## Auf Ordner zugreifen und daten in einem DF speichern

In [3]:
# Base path to archive Folder
base_path = 'archive'

# List of all seasonal folders in the archive
season_folders = [folder for folder in os.listdir(base_path) if os.path.isdir(os.path.join(base_path, folder))]

# sort folder by year  
season_folders.sort()

# For each season, load the Excel file and save it in a list of DataFrames
dataframes = []
for season in season_folders:
    file_path = os.path.join(base_path, season, 'champs.csv')
    df = pd.read_csv(file_path)
    df['season'] = season  # optional: add a new column "season" to save each season
    dataframes.append(df)

# Combine all Dataframes into a single DataFrame
combined_df = pd.concat(dataframes, ignore_index=True)
print(combined_df)


                   Round                     Date  \
0        Round 1 | Leg 1   (Sun) 4 Sep 1955 (W35)   
1        Round 1 | Leg 1   (Wed) 7 Sep 1955 (W36)   
2        Round 1 | Leg 1   (Thu) 8 Sep 1955 (W36)   
3        Round 1 | Leg 1  (Wed) 14 Sep 1955 (W37)   
4        Round 1 | Leg 1  (Tue) 20 Sep 1955 (W38)   
...                  ...                      ...   
6549  Semifinals | Leg 1  (Tue) 26 Apr 2016 (W17)   
6550  Semifinals | Leg 1  (Wed) 27 Apr 2016 (W17)   
6551  Semifinals | Leg 2   (Tue) 3 May 2016 (W18)   
6552  Semifinals | Leg 2   (Wed) 4 May 2016 (W18)   
6553               Final  (Sat) 28 May 2016 (W21)   

                                Team 1       FT   HT  \
0                Sporting CP › POR (1)      3-3  1-1   
1     Budapesti Voros Lobogo › HUN (1)      6-3  3-2   
2            Servette Geneve › SUI (1)      0-2  0-0   
3            Rot-Weiss Essen › GER (1)      0-4  0-2   
4             Djurgardens IF › SWE (1)      0-0  0-0   
...                        

## "Group" und "Comments" entfernen

In [4]:
# List of columns to be removed
columns_to_drop = ['Comments', 'Stage']

# Removing the columns from the DataFrame
combined_df = combined_df.drop(columns=columns_to_drop)

## Neue Zeile für Verlängerung

In [8]:
def check_overtime(row):
    overtime = 'no'
    # Edit the "FT" column
    if re.search(r'\(\*\)', row['FT']):
        row['FT'] = re.sub(r' \(\*\)', '', row['FT'])
        overtime = 'yes'
    
    # Edit the "ET" column
    if pd.notna(row['ET']) and re.search(r' \(a\.e\.t\.\)', row['ET']):
        row['ET'] = re.sub(r' \(a\.e\.t\.\)', '', row['ET'])
        overtime = 'yes'
    
    # Edit the "P" column
    if pd.notna(row['P']) and re.search(r' \(pen\.\)', row['P']):
        row['P'] = re.sub(r' \(pen\.\)', '', row['P'])
        overtime = 'penalty'
    
    return overtime

# Update the columns "FT", "ET", and "P" and create the new column "Extra Time"
combined_df['Overtime'] = combined_df.apply(check_overtime, axis=1)
print(combined_df.head())

             Round                     Date                            Team 1  \
0  Round 1 | Leg 1   (Sun) 4 Sep 1955 (W35)             Sporting CP › POR (1)   
1  Round 1 | Leg 1   (Wed) 7 Sep 1955 (W36)  Budapesti Voros Lobogo › HUN (1)   
2  Round 1 | Leg 1   (Thu) 8 Sep 1955 (W36)         Servette Geneve › SUI (1)   
3  Round 1 | Leg 1  (Wed) 14 Sep 1955 (W37)         Rot-Weiss Essen › GER (1)   
4  Round 1 | Leg 1  (Tue) 20 Sep 1955 (W38)          Djurgardens IF › SWE (1)   

    FT   HT                       Team 2  ∑FT   ET    P   season Group  \
0  3-3  1-1  Partizan Belgrade › SRB (1)  NaN  NaN  NaN  1955-56   NaN   
1  6-3  3-2     RSC Anderlecht › BEL (1)  NaN  NaN  NaN  1955-56   NaN   
2  0-2  0-0     Real Madrid CF › ESP (1)  NaN  NaN  NaN  1955-56   NaN   
3  0-4  0-2       Hibernian FC › SCO (1)  NaN  NaN  NaN  1955-56   NaN   
4  0-0  0-0   Gwardia Warszawa › POL (1)  NaN  NaN  NaN  1955-56   NaN   

  Verlängerung  
0         nein  
1         nein  
2         nein  


## Neue Spalte für einzelne Tore

In [9]:
def get_goals(row):
    for col in ['P', 'ET', 'FT']:
        if pd.notna(row[col]):
            return row[col].split('-')

combined_df[['Goals hometeam', 'Goals guestteam']] = combined_df.apply(get_goals, axis=1, result_type='expand').astype(int)
print(combined_df.head())

             Round                     Date                            Team 1  \
0  Round 1 | Leg 1   (Sun) 4 Sep 1955 (W35)             Sporting CP › POR (1)   
1  Round 1 | Leg 1   (Wed) 7 Sep 1955 (W36)  Budapesti Voros Lobogo › HUN (1)   
2  Round 1 | Leg 1   (Thu) 8 Sep 1955 (W36)         Servette Geneve › SUI (1)   
3  Round 1 | Leg 1  (Wed) 14 Sep 1955 (W37)         Rot-Weiss Essen › GER (1)   
4  Round 1 | Leg 1  (Tue) 20 Sep 1955 (W38)          Djurgardens IF › SWE (1)   

    FT   HT                       Team 2  ∑FT   ET    P   season Group  \
0  3-3  1-1  Partizan Belgrade › SRB (1)  NaN  NaN  NaN  1955-56   NaN   
1  6-3  3-2     RSC Anderlecht › BEL (1)  NaN  NaN  NaN  1955-56   NaN   
2  0-2  0-0     Real Madrid CF › ESP (1)  NaN  NaN  NaN  1955-56   NaN   
3  0-4  0-2       Hibernian FC › SCO (1)  NaN  NaN  NaN  1955-56   NaN   
4  0-0  0-0   Gwardia Warszawa › POL (1)  NaN  NaN  NaN  1955-56   NaN   

  Verlängerung  Tore Heimteam  Tore Auswärtsteam  
0         nein   

## Web application with streamlit

In [26]:
st.set_page_config(page_title='Champions League', page_icon=':trophy:', layout='wide',)
st.title('Champions League')

# Create a selectbox to choose from the names in the DataFrame
selected_season = st.selectbox('Select a season', combined_df['season'].unique())

# Filter the DataFrame based on the selected name
filtered_df = combined_df[combined_df['season'] == selected_season]
st.write(filtered_df)

# Calculate total number of games, goals and average goals
total_games = len(filtered_df)
total_goals = int(filtered_df['Goals hometeam'].sum() + filtered_df['Goals guestteam'].sum())
avgGoals = round((total_goals/total_games), 2)

# Display the statistics
st.write(f"Total number of games for the {selected_season} season: **{total_games}**")
st.write(f"Total number of goals for the {selected_season} season: **{total_goals}**")
st.write(f"Average goals per game: **{avgGoals}**")

2023-05-24 13:10:05.892 
  command:

    streamlit run c:\Users\pasca\anaconda3\envs\spenv\lib\site-packages\ipykernel_launcher.py [ARGUMENTS]


## Footer

In [25]:
import os
import platform
import socket
from platform import python_version
from datetime import datetime

print('-----------------------------------')
print(os.name.upper())
print(platform.system(), '|', platform.release())
print('Datetime:', datetime.now().strftime("%Y-%m-%d %H:%M:%S"))
print('Python Version:', python_version())
print('-----------------------------------')

-----------------------------------
NT
Windows | 10
Datetime: 2023-05-24 13:08:19
Python Version: 3.10.9
-----------------------------------
